## Centrar los datos, under sampling y over sampling##


In [1]:
#Imports
import pandas as pd
import numpy as np

from sklearn.metrics import SCORERS
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import DistanceMetric

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate


from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek

import xgboost as xgb
from xgboost.sklearn import XGBClassifier

from sklearn import preprocessing


In [40]:
tf = pd.read_csv('featuresMagui.csv') #mis features 
tf_seba = pd.read_csv('features_seba.csv') #los features de Seba
labels = pd.read_csv('../data/labels_training_set.csv') #las personas de las cuales tengo Info
personas =pd.read_csv('../data/trocafone_kaggle_test.csv') #las personas a las que le tengo que predecir

In [3]:
#Datos a entregar
datos = pd.merge(tf, tf_seba, on = 'person', how = 'inner')
datos = pd.merge(personas, datos, on = 'person', how = 'inner')

In [4]:
#Datos para entrenar
labels_f = pd.merge(tf,tf_seba, on = 'person',how = 'inner')
labels_f = pd.merge(labels, labels_f, on = 'person', how = 'inner')

In [5]:
columnas = labels_f.columns

In [6]:
#Defino Xgboost en un solo, lo que hace es entrenar predecir e imprimir las predicciones. 

In [7]:
def xgboost(X_p, y_p, X_train_p, X_test_p, y_train_p, y_test_p):
    xg_reg = xgb.XGBClassifier(learning_rate =0.05,n_estimators=1000,max_depth=5,min_child_weight=1,gamma=0,
                           subsample=0.8,colsample_bytree=0.8,objective= 'binary:logistic',nthread=4,
                            scale_pos_weight=1, seed=27)
    xg_reg.fit(X_train_p,y_train_p, early_stopping_rounds = 5, eval_set=[(X_test, y_test)])
    preds = xg_reg.predict(X_test_p)
    preds_prob = xg_reg.predict_proba(X_test_p)[:,1]
    train_accuracy = accuracy_score(y_train_p, xg_reg.predict(X_train_p))
    test_accuracy = accuracy_score(y_test_p, preds)
    area_debajo_de_curva = roc_auc_score(y_test_p, preds_prob)
    matriz_de_confusion = confusion_matrix(y_test_p, preds)
    print('train acurracy: ')
    print(train_accuracy)
    print('test acurracy: ')
    print(test_accuracy)
    print('Matriz de confusión: ')
    print(matriz_de_confusion)
    print('Área bajo la curva: ')
    print(area_debajo_de_curva)
    return(xg_reg)



In [8]:
print(columnas)

Index(['person', 'label', 'Unnamed: 0', 'esNuevo', 'cantidadDeVisitas',
       'ingresoPorPublicidad', 'Facebook', 'FacebookAds', 'FacebookSocial',
       'MARKETING SOCIAL',
       ...
       'viewed product mes 4', 'visited site mes 4', 'distan_dias',
       'nuevo_mes_5', 'cant_dias_dist', 'modelos_dist', 'promedio_por_dia_x',
       'promedio_por_mes', 'mes_primer_entrada', 'promedio_por_dia_y'],
      dtype='object', length=138)


In [9]:
#Por un lado tengo los features y por otro los labels. 
labels_f.reset_index()

y = labels_f.iloc[:,1:2]
X = labels_f.iloc[:,3: 138]

In [10]:
X = X.drop('device_type', axis = 1)

In [11]:
#for columna in X.columns: 
 #   print(columna)

Aplico XGboost solo, es decir, **sin ninguna mejora **

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.1, random_state= 123)

xg_reg =xgboost(X, y, X_train, X_test, y_train, y_test)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0]	validation_0-error:0.049949
Will train until validation_0-error hasn't improved in 5 rounds.
[1]	validation_0-error:0.049949
[2]	validation_0-error:0.049434
[3]	validation_0-error:0.049949
[4]	validation_0-error:0.048919
[5]	validation_0-error:0.047889
[6]	validation_0-error:0.047889
[7]	validation_0-error:0.048404
[8]	validation_0-error:0.048404
[9]	validation_0-error:0.048404
[10]	validation_0-error:0.048404
Stopping. Best iteration:
[5]	validation_0-error:0.047889

train acurracy: 
0.9508928571428571
test acurracy: 
0.9521112255406797
Matriz de confusión: 
[[1847    1]
 [  92    2]]
Área bajo la curva: 
0.8188870083816893


Los datos de la matriz de confusión son **ALARMANTES** ya que predice mal los que compran. 

Trato de centrar los datos con el **escalador estandar** 

In [13]:
scaler = preprocessing.StandardScaler().fit(X)
X_transformed = scaler.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size= 0.1, random_state= 123)

xg_reg =xgboost(X_transformed,y,X_train, X_test, y_train, y_test)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0]	validation_0-error:0.049949
Will train until validation_0-error hasn't improved in 5 rounds.
[1]	validation_0-error:0.049949
[2]	validation_0-error:0.049434
[3]	validation_0-error:0.049949
[4]	validation_0-error:0.048919
[5]	validation_0-error:0.047889
[6]	validation_0-error:0.047889
[7]	validation_0-error:0.048404
[8]	validation_0-error:0.048404
[9]	validation_0-error:0.048404
[10]	validation_0-error:0.048404
Stopping. Best iteration:
[5]	validation_0-error:0.047889

train acurracy: 
0.9508928571428571
test acurracy: 
0.9521112255406797
Matriz de confusión: 
[[1847    1]
 [  92    2]]
Área bajo la curva: 
0.8188870083816893


Trato de centrar los datos con el escalador **MaxAbsScaler **

In [14]:
scaler = preprocessing.MaxAbsScaler().fit(X)
X_transformed = scaler.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size= 0.1, random_state= 123)

xg_reg =xgboost(X_transformed, y, X_train, X_test, y_train, y_test)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0]	validation_0-error:0.049949
Will train until validation_0-error hasn't improved in 5 rounds.
[1]	validation_0-error:0.049949
[2]	validation_0-error:0.049434
[3]	validation_0-error:0.049949
[4]	validation_0-error:0.048919
[5]	validation_0-error:0.047889
[6]	validation_0-error:0.047889
[7]	validation_0-error:0.048404
[8]	validation_0-error:0.048404
[9]	validation_0-error:0.048404
[10]	validation_0-error:0.048404
Stopping. Best iteration:
[5]	validation_0-error:0.047889

train acurracy: 
0.9508928571428571
test acurracy: 
0.9521112255406797
Matriz de confusión: 
[[1847    1]
 [  92    2]]
Área bajo la curva: 
0.8188870083816893


El escalador MaxAbsScales es mejor que el estandar. 

En función de los datos obtenidos de la matriz de confusión, nos dimos cuenta que estaba dando muchos falsos negativos de los que compraron. Por lo tanto, recurriremos a una técnica de **UNDER SAMPLING** y luego a otra técnica de **OVER SAMPLING** y luego compararemos los resultados obtenidos.

Uso otra manera de hacer over\under sampling que es mediante la librería **imbalanced-learn** de python

INFO: 

For example, we can cluster the records of the majority class, and do the under-sampling by removing records from each cluster, thus seeking to preserve information. In over-sampling, instead of creating exact copies of the minority class records, we can introduce small variations into those copies, creating more diverse synthetic samples.



In [15]:
#UNDER SAMPLING

In [37]:

rus= RandomUnderSampler(return_indices=True)

#id_rus son los índices
X_rus_train, y_rus_train, id_rus = rus.fit_sample(X_train, y_train)
print(X_rus_train)
xg_reg = xgboost(X_transformed, y ,X_rus_train, X_test, y_rus_train, y_test)

[[0.         0.00680272 1.         ... 0.01211933 0.4        0.09593023]
 [0.         0.01360544 0.         ... 0.00341827 0.8        0.00872093]
 [1.         0.00340136 0.         ... 0.00124301 1.         0.00581395]
 ...
 [0.         0.03401361 1.         ... 0.03791175 0.6        0.04844961]
 [0.         0.02040816 1.         ... 0.03449347 0.4        0.09496124]
 [0.         0.03741497 1.         ... 0.12057178 0.8        0.15863787]]
[0]	validation_0-error:0.327497
Will train until validation_0-error hasn't improved in 5 rounds.
[1]	validation_0-error:0.23069
[2]	validation_0-error:0.235324
[3]	validation_0-error:0.236869
[4]	validation_0-error:0.221421
[5]	validation_0-error:0.232235
[6]	validation_0-error:0.22863
[7]	validation_0-error:0.234295
[8]	validation_0-error:0.238929
[9]	validation_0-error:0.238414
Stopping. Best iteration:
[4]	validation_0-error:0.221421



/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


train acurracy: 
0.8177200902934537
test acurracy: 
0.7785787847579815
Matriz de confusión: 
[[1437  411]
 [  19   75]]
Área bajo la curva: 
0.8417179008934328


In [46]:
datos_f = datos.iloc[:,2:138]
#print(datos_f)
datos_f = datos_f.drop('device_type', axis= 1)
scaler = preprocessing.MaxAbsScaler().fit(datos_f)
datos_f = scaler.transform(datos_f)
#print(datos_f)
preds_entrega = xg_reg.predict_proba(datos_f)[:,1]

In [47]:
preds_entrega

array([0.43222865, 0.5342236 , 0.442458  , ..., 0.5310635 , 0.5310635 ,
       0.5310635 ], dtype=float32)

In [48]:
preds = pd.DataFrame(preds_entrega)

In [49]:
df_final = datos[['person']].copy()

In [50]:
df_final['label'] = preds[0]

In [51]:
print(df_final)

         person     label
0      4886f805  0.432229
1      0297fc1e  0.534224
2      2d681dd8  0.442458
3      cccea85e  0.536295
4      4c8a8b93  0.514588
5      29ebb414  0.453225
6      3dc1950f  0.529518
7      8ea4c165  0.475712
8      d8cfe234  0.463914
9      d6bc64df  0.475712
10     ec9c4059  0.478588
11     d21b8e6b  0.411343
12     2a724d87  0.432219
13     686c49c9  0.549183
14     a08d42ea  0.489756
15     c98f5cf1  0.498367
16     d614c608  0.432229
17     e45acd53  0.502977
18     7d876393  0.481477
19     f5af843f  0.483147
20     5a724794  0.510297
21     00091926  0.493040
22     55d1e0ee  0.486876
23     f87be219  0.421760
24     49c19e32  0.459079
25     bb78c182  0.432219
26     e2bfe05f  0.432219
27     0e9d0ae2  0.423115
28     cb68850c  0.448520
29     f30ef764  0.441335
...         ...       ...
19385  523c7e69  0.528681
19386  d2e564cb  0.521841
19387  eb27e544  0.502179
19388  b32e7113  0.528681
19389  a65f2cf0  0.528681
19390  a70b9f00  0.482477
19391  a161f

In [52]:
df_final.to_csv('predicciones_magui.csv', encoding='utf-8', index=False)

In [98]:
#OVER SAMPLING

In [99]:
ros = RandomOverSampler()
X_ros_train, y_ros_train = ros.fit_sample(X_train, y_train)

xgboost(X_transformed, y, X_ros_train, X_test, y_ros_train, y_test)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0]	validation_0-error:0.231205
Will train until validation_0-error hasn't improved in 5 rounds.
[1]	validation_0-error:0.227085
[2]	validation_0-error:0.225026
[3]	validation_0-error:0.233265
[4]	validation_0-error:0.241504
[5]	validation_0-error:0.22863
[6]	validation_0-error:0.238929
[7]	validation_0-error:0.238414
Stopping. Best iteration:
[2]	validation_0-error:0.225026

train acurracy: 
0.8171952248884602
test acurracy: 
0.7749742533470649
Matriz de confusión: 
[[1434  414]
 [  23   71]]
Área bajo la curva: 
0.8331289720917381


Uso otra manera de hacer Undersample (que estuvo dando buenos resultados) con la siguiente técnica **Under-sampling: Tomek links **

Info:

Tomek links are pairs of very close instances, but of opposite classes. Removing the instances of the majority class of each pair increases the space between the two classes, facilitating the classification process.

In [100]:
tl = TomekLinks(return_indices=True, ratio='majority')

X_tl_train, y_tl_train, id_tl = tl.fit_sample(X_train, y_train)

xgboost(X_transformed, y, X_tl_train, X_test, y_tl_train, y_test)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0]	validation_0-error:0.053038
Will train until validation_0-error hasn't improved in 5 rounds.
[1]	validation_0-error:0.050463
[2]	validation_0-error:0.049949
[3]	validation_0-error:0.049949
[4]	validation_0-error:0.049434
[5]	validation_0-error:0.049949
[6]	validation_0-error:0.049434
[7]	validation_0-error:0.049434
[8]	validation_0-error:0.049434
[9]	validation_0-error:0.049434
Stopping. Best iteration:
[4]	validation_0-error:0.049434

train acurracy: 
0.9498076250437216
test acurracy: 
0.9505664263645726
Matriz de confusión: 
[[1846    2]
 [  94    0]]
Área bajo la curva: 
0.8154848254582296


Uso otra manera de hacer Under sample que es **Cluster Centroids**.

INFO:

This technique performs under-sampling by generating centroids based on clustering methods. The data will be previously grouped by similarity, in order to preserve information.


In [101]:
cc = ClusterCentroids(ratio={0: 10})
X_cc_train, y_cc_train = cc.fit_sample(X_train, y_train)


xgboost(X_transformed, y, X_cc_train, X_test, y_cc_train, y_test)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0]	validation_0-error:0.951596
Will train until validation_0-error hasn't improved in 5 rounds.
[1]	validation_0-error:0.951596
[2]	validation_0-error:0.951596
[3]	validation_0-error:0.951596
[4]	validation_0-error:0.951596
[5]	validation_0-error:0.951596
Stopping. Best iteration:
[0]	validation_0-error:0.951596

train acurracy: 
0.9988839285714286
test acurracy: 
0.04840370751802266
Matriz de confusión: 
[[   0 1848]
 [   0   94]]
Área bajo la curva: 
0.5


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Otra manera de hacer Over Sampling es **SMOTE**.

Info: 

SMOTE (Synthetic Minority Oversampling TEchnique) consists of synthesizing elements for the minority class, based on those that already exist. It works randomly picking a point from the minority class and computing the k-nearest neighbors for this point. The synthetic points are added between the chosen point and its neighbors.

In [102]:
smote = SMOTE(ratio='minority')
X_sm_train, y_sm_train = smote.fit_sample(X_train, y_train)

xgboost(X_transformed, y, X_sm_train, X_test, y_sm_train, y_test)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0]	validation_0-error:0.141092
Will train until validation_0-error hasn't improved in 5 rounds.
[1]	validation_0-error:0.136457
[2]	validation_0-error:0.133368
[3]	validation_0-error:0.211123
[4]	validation_0-error:0.181771
[5]	validation_0-error:0.170958
[6]	validation_0-error:0.167868
[7]	validation_0-error:0.166838
Stopping. Best iteration:
[2]	validation_0-error:0.133368

train acurracy: 
0.8648559025684313
test acurracy: 
0.8666323377960865
Matriz de confusión: 
[[1638  210]
 [  49   45]]
Área bajo la curva: 
0.797886732062264


La última manera que voy a realizar sampling es una **combinación entre SMOTE y Tomek Link**.

In [103]:
smt = SMOTETomek(ratio='auto')
X_smt_train, y_smt_train = smt.fit_sample(X_train, y_train)

xgboost(X_transformed, y, X_smt_train, X_test, y_smt_train, y_test)


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0]	validation_0-error:0.133368
Will train until validation_0-error hasn't improved in 5 rounds.
[1]	validation_0-error:0.143151
[2]	validation_0-error:0.210608
[3]	validation_0-error:0.213697
[4]	validation_0-error:0.211637
[5]	validation_0-error:0.173018
Stopping. Best iteration:
[0]	validation_0-error:0.133368

train acurracy: 
0.8523536511768256
test acurracy: 
0.8666323377960865
Matriz de confusión: 
[[1636  212]
 [  47   47]]
Área bajo la curva: 
0.7790624712167267


## Para hacer la entrega en Kaggle ##

In [33]:
datos_f = datos.loc[:, columnas]
scaler = preprocessing.MaxAbsScaler().fit(datos_f)
datos_f = scaler.transform(datos_f)
preds_entrega = xg_reg.predict_proba(datos_f)[:,1]

In [34]:
preds_entrega

array([0.99993944, 0.9332026 , 0.99999094, ..., 0.15517399, 0.05298465,
       0.05298465], dtype=float32)

In [35]:
preds = pd.DataFrame(preds_entrega)

In [38]:
df_final = datos[['person']].copy()

In [40]:
df_final['label'] = preds[0]

In [43]:
df_final.to_csv('predicciones_magui.csv', encoding='utf-8', index=False)

ESTO DE ACÁ ABAJO NO ANDA

In [ ]:
#print(X.info())
X['esNuevo']= X['esNuevo'].astype(int)
X['adCampaignHit']= X['adCampaignHit'].astype(int)           
X['bandListing']= X['bandListing'].astype(int)              
X['checkout']=   X['checkout'].astype(int)                
X['genericListing']=  X['genericListing'].astype(int)       
X['searchEngineHit']= X['searchEngineHit'].astype(int)         
X['searchedProducts']= X['searchedProducts'].astype(int)          
X['staticpage']= X['staticpage'].astype(int)               
X['viewedProduct']= X['viewedProduct'].astype(int)             
X['visitedSite']= X['visitedSite'].astype(int)               
X['deviceSmartphone']= X['deviceSmartphone'].astype(int)          
X['deviceComputer']= X['deviceComputer'].astype(int)           
X['deviceTablet']= X['deviceTablet'].astype(int)              
X['deviceUnknown']= X['deviceUnknown'].astype(int)             
X['visitaALaManiana']= X['visitaALaManiana'].astype(int)         
X['visitaALaNoche']= X['visitaALaNoche'].astype(int)          
X['visitaALaTarde']=  X['visitaALaTarde'].astype(int)           
X['visitaALaMadrugada']= X['visitaALaMadrugada'].astype(int)
#print(X.info())

In [13]:
#Decido usar el MaxAbsScaler
scaler = preprocessing.MaxAbsScaler().fit(X)
X_transformed = scaler.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size= 0.1, random_state= 123)

In [14]:
#UNDER SAMPLING
#cuánta información hay de cada clase 
count_class_0, count_class_1 = X_train.label_x.value_counts()

print(count_class_0, count_class_1)
# divido por clase la información
label_0 = X_train[X_train['label_x'] == 0]
label_1 = X_train[X_train['label_x'] == 1]

AttributeError: 'numpy.ndarray' object has no attribute 'label_x'

In [ ]:
label_0_under = label_0.sample(count_class_1)
label_f_under = pd.concat([label_0_under, label_1], axis=0)

X_train_u = label_f_under.loc[:,columnas]
y_train_u = label_f_under.loc[:,['label_x']]


xgboost(X_transformed, y, X_train_u, X_test, y_train_u, y_test)

In [ ]:
#OVER SAMPLING
count_class_0, count_class_1 = X_train.label_x.value_counts()

label_0 = X_train[X_train['label_x'] == 0]
label_1 = X_train[X_train['label_x'] == 1]

In [ ]:
label_1_over = label_1.sample(count_class_0, replace=True)
label_f_over = pd.concat([label_0, label_1_over], axis=0)

X_train_o = X_train.loc[:,columnas]
y_train_o = X_train.loc[:,['label_x']]

xgboost(X_transformed, y, X_train_o, X_test, y_train_o, y_test)

Como podemos ver usando **Under sampling, mejoraron los resultados** teniendo en cuenta la matriz de confusión, pero **Over sampling** es mucho mejor.